In [1]:
import { TireProperties } from '../../src/tire-properties.js'
import { Pacejka2002 } from '../../src/tire-models/pac2002.js'
import * as Plot from 'npm:@observablehq/plot'
import { DOMParser, SVGElement } from 'npm:linkedom'

In [2]:
function genRange(start, stop, step) {
  const range = []
  for (let i = start; i <= stop; i += step) {
    range.push(i)
  }
  return range
}

function toDegrees(angle) {
  return angle * (180 / Math.PI)
}

function toRadians(angle) {
  return angle * (Math.PI / 180)
}

In [3]:
const document = new DOMParser().parseFromString(
  `<!DOCTYPE html><html lang="en"></html>`,
  'text/html',
)

In [4]:
const properties = new TireProperties()
properties.fromTirFile(
  await Deno.readTextFileSync('../../tests/fixtures/audi.tir'),
)
const pac2002Model = new Pacejka2002({
  tireProperties: properties,
})
const modelInitializationErrors = pac2002Model.initializeFromProperties()

In [5]:
const latForceSlipAngles = genRange(toRadians(-30), toRadians(30), toRadians(1))
const latForceData = latForceSlipAngles.map((angleRad) => {
  const outputVector = [0, 0, 0, 0]
  pac2002Model.computeLateralForce(
    angleRad,
    pac2002Model.verticalParameters.fNomin,
    0,
    1.0,
    outputVector,
  )
  return {
    angleDeg: toDegrees(angleRad),
    latForce: outputVector[0],
  }
})

In [6]:
const slipRatios = genRange(-1, 1, 0.01)
const longForceData = slipRatios.map((slipRatio) => {
  return {
    slipRatio: slipRatio,
    longForce: pac2002Model.computeLongitudinalForce(
      slipRatio,
      pac2002Model.verticalParameters.fNomin,
      0,
      1.0,
    ),
  }
})

In [7]:
const aligningMomentSlipAngles = genRange(
  toRadians(-30),
  toRadians(30),
  toRadians(1),
)
const aligningMomentData = aligningMomentSlipAngles.map((angleRad) => {
  const latForceOutputVector = [0, 0, 0, 0]
  pac2002Model.computeLateralForce(
    angleRad,
    pac2002Model.verticalParameters.fNomin,
    0,
    1.0,
    latForceOutputVector,
  )
  return {
    angleDeg: toDegrees(angleRad),
    aligningMoment: pac2002Model.computeAligningMoment(
      angleRad,
      pac2002Model.verticalParameters.fNomin,
      0,
      ...latForceOutputVector,
    ),
  }
})

In [8]:
Plot.plot({
  title: 'Audi Tire',
  x: {
    label: 'Slip angle (Degrees)',
  },
  y: {
    label: 'Tire lateral force (Newtons)',
    grid: true,
  },
  marks: [
    Plot.ruleY([0]),
    Plot.line(latForceData, { x: 'angleDeg', y: 'latForce' }),
  ],
  document,
})

Audi Tire −5,000 −4,000 −3,000 −2,000 −1,000 0 1,000 2,000 3,000 4,000 5,000 6,000 ↑ Tire lateral force (Newtons) −20 −10 0 10 20 Slip angle (Degrees) → <path d="M40,20L49.831,20L59.661,20L69.492,20L79.322,20L89.153,20L98.983,20L108.814,20L118.644,20L128.475,20L138.305,20L148.136,20L157.966,20L167.797,20L177.627,20L187.458,20L197.288,20L207.119,20L216.949,20L226.78,20L236.61,20L246.441,22.395L256.271,26.705L266.102,32.958L275.932,41.912L285.763,54.554L295.593,72.051L305.424,95.526L315.254,125.54L325.085,161.332L334.915,200.339L344.746,238.815L354.576,273.388L364.407,301.997L374.237,324.134L384.068,340.418L393.898,351.962L403.729,359.923L413.559,365.283L423.39,368.8L433.22,370L443.051,370L452.881,370L462.712,370L472.542,370L482.373,370L492.203,370L502.034,370L511.864,370L521.695,370L531.525,370L541.356,370L551.186,370L561.017,370L570.847,370L580.678,370L590.508,370L600.339,370L610.169,370L620,370" />

In [9]:
Plot.plot({
  title: 'Audi Tire',
  x: {
    label: 'Slip ratio',
  },
  y: {
    label: 'Tire longitudinal force (Newtons)',
    grid: true,
  },
  marks: [
    Plot.ruleY([0]),
    Plot.line(longForceData, { x: 'slipRatio', y: 'longForce' }),
  ],
  document,
})

Audi Tire −6,000 −5,000 −4,000 −3,000 −2,000 −1,000 0 1,000 2,000 3,000 4,000 5,000 6,000 ↑ Tire longitudinal force (Newtons) −1.0 −0.8 −0.6 −0.4 −0.2 0.0 0.2 0.4 0.6 0.8 Slip ratio → <path d="M40,328.11L42.915,328.402L45.829,328.698L48.744,328.998L51.658,329.302L54.573,329.611L57.487,329.924L60.402,330.242L63.317,330.565L66.231,330.892L69.146,331.224L72.06,331.562L74.975,331.904L77.889,332.251L80.804,332.604L83.719,332.962L86.633,333.325L89.548,333.695L92.462,334.07L95.377,334.45L98.291,334.837L101.206,335.23L104.121,335.629L107.035,336.034L109.95,336.446L112.864,336.865L115.779,337.29L118.693,337.722L121.608,338.162L124.523,338.608L127.437,339.062L130.352,339.523L133.266,339.992L136.181,340.468L139.095,340.953L142.01,341.446L144.925,341.946L147.839,342.456L150.754,342.973L153.668,343.5L156.583,344.035L159.497,344.579L162.412,345.133L165.327,345.695L168.241,346.267L171.156,346.848L174.07,347.439L176.985,348.04L179.899,348.65L182.814,349.271L185.729,349.901L188.643,350.541L191.558,351.191L194.472,351.851L197.387,352.521L200.302,353.2L203.216,353.889L206.131,354.588L209.045,355.295L211.96,356.011L214.874,356.735L217.789,357.467L220.704,358.205L223.618,358.95L226.533,359.699L229.447,360.452L232.362,361.208L235.276,361.963L238.191,362.717L241.106,363.466L244.02,364.208L246.935,364.938L249.849,365.653L252.764,366.346L255.678,367.013L258.593,367.644L261.508,368.23L264.422,368.761L267.337,369.222L270.251,369.597L273.166,369.865L276.08,370L278.995,369.971L281.91,369.737L284.824,369.249L287.739,368.444L290.653,367.241L293.568,365.538L296.482,363.205L299.397,360.074L302.312,355.934L305.226,350.518L308.141,343.495L311.055,334.473L313.97,323.011L316.884,308.662L319.799,291.059L322.714,270.051L325.628,245.858L328.543,219.196L331.457,191.258L334.372,163.522L337.286,137.418L340.201,114.025L343.116,93.92L346.03,77.21L348.945,63.671L351.859,52.903L354.774,44.454L357.688,37.895L360.603,32.848L363.518,29L366.432,26.1L369.347,23.95L372.261,22.391L375.176,21.303L378.09,20.59L381.005,20.175L383.92,20L386.834,20.018L389.749,20.191L392.663,20.49L395.578,20.889L398.492,21.371L401.407,21.918L404.322,22.517L407.236,23.158L410.151,23.832L413.065,24.532L415.98,25.251L418.894,25.985L421.809,26.729L424.724,27.48L427.638,28.234L430.553,28.99L433.467,29.745L436.382,30.497L439.296,31.245L442.211,31.988L445.126,32.725L448.04,33.455L450.955,34.177L453.869,34.891L456.784,35.596L459.698,36.291L462.613,36.978L465.528,37.655L468.442,38.322L471.357,38.979L474.271,39.627L477.186,40.264L480.101,40.892L483.015,41.51L485.93,42.117L488.844,42.716L491.759,43.304L494.673,43.883L497.588,44.452L500.503,45.012L503.417,45.563L506.332,46.105L509.246,46.638L512.161,47.162L515.075,47.677L517.99,48.184L520.905,48.683L523.819,49.173L526.734,49.656L529.648,50.13L532.563,50.597L535.477,51.056L538.392,51.508L541.307,51.953L544.221,52.39L547.136,52.82L550.05,53.244L552.965,53.66L555.879,54.071L558.794,54.474L561.709,54.872L564.623,55.263L567.538,55.648L570.452,56.027L573.367,56.401L576.281,56.768L579.196,57.131L582.111,57.487L585.025,57.838L587.94,58.184L590.854,58.525L593.769,58.861L596.683,59.192L599.598,59.518L602.513,59.839L605.427,60.156L608.342,60.468L611.256,60.776L614.171,61.079L617.085,61.378L620,61.673" />

In [10]:
Plot.plot({
  title: 'Audi Tire',
  x: {
    label: 'Slip angle (Degrees)',
  },
  y: {
    label: 'Aligning moment (Newton-meters)',
    grid: true,
  },
  marks: [
    Plot.ruleY([0]),
    Plot.line(aligningMomentData, { x: 'angleDeg', y: 'aligningMoment' }),
  ],
  document,
})

Audi Tire −600 −500 −400 −300 −200 −100 0 100 200 300 400 500 600 ↑ Aligning moment (Newton-meters) −20 −10 0 10 20 Slip angle (Degrees) → <path d="M40,345.89L49.831,347.509L59.661,349.084L69.492,350.613L79.322,352.096L89.153,353.534L98.983,354.927L108.814,356.274L118.644,357.577L128.475,358.834L138.305,360.048L148.136,361.218L157.966,362.345L167.797,363.431L177.627,364.476L187.458,365.482L197.288,366.45L207.119,367.384L216.949,368.285L226.78,369.156L236.61,370L246.441,368.418L256.271,364.879L266.102,359.341L275.932,351.031L285.763,338.935L295.593,321.848L305.424,298.598L315.254,268.575L325.085,232.511L334.915,192.992L344.746,153.839L354.576,118.527L364.407,89.217L374.237,66.487L384.068,49.755L393.898,37.913L403.729,29.8L413.559,24.419L423.39,20.998L433.22,20L443.051,20.288L452.881,20.655L462.712,21.098L472.542,21.613L482.373,22.199L492.203,22.851L502.034,23.568L511.864,24.349L521.695,25.19L531.525,26.091L541.356,27.05L551.186,28.066L561.017,29.138L570.847,30.264L580.678,31.443L590.508,32.676L600.339,33.96L610.169,35.296L620,36.681" />